In [24]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

emotion_folders = ['angry', 'fear', 'happy', 'sad']
data = []
for emotion in emotion_folders:
    folder_path = os.path.join(emotion)
    files = os.listdir(folder_path)
    for file in files:
        data.append([file, emotion])

df = pd.DataFrame(data, columns=['file', 'emotion'])
train_ratio = 0.7
train_df, test_df = train_test_split(df, train_size=train_ratio, random_state=42)
train_test_folders = ['train', 'test']

for emotion in emotion_folders:
    for tt in train_test_folders:
        directory = os.path.join( f'{emotion}_{tt}')
        if not os.path.exists(directory):
            os.makedirs(directory)

for index, row in train_df.iterrows():
    os.rename(os.path.join( row['emotion'], row['file']), os.path.join( f'{row["emotion"]}_train', row['file']))

for index, row in test_df.iterrows():
    os.rename(os.path.join( row['emotion'], row['file']), os.path.join( f'{row["emotion"]}_test', row['file']))

[]
